In [1]:
from sklearn.datasets import load_files 
import keras
from keras.utils import np_utils
import numpy as np
from glob import glob

image_login_paths = [(lambda i: i[:-1])(i) for i in open('./Images/Paths/path_login.txt')]
image_non_login_paths = [(lambda i: i[:-1])(i) for i in open('./Images/Paths/path_non_login.txt')]

X = image_login_paths + image_non_login_paths
y = [1]*len(image_login_paths) + [0]*len(image_non_login_paths)

Using TensorFlow backend.


In [2]:
## Splitting the data into training and testing sets ##
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size=0.8, shuffle=True, 
                                                    random_state=42)

In [3]:
## Preprocessing the Data ##
from keras.preprocessing import image  
from PIL import Image
# from tqdm import tqdm
import tensorflow as tf
import io

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = Image.open(img_path).convert('LA')
    img = img.resize((800, 600))
    x = image.img_to_array(img)
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in img_paths]
    return np.vstack(list_of_tensors)

In [4]:
from PIL import ImageFile                            
# pre-process the data for Keras
train_tensors = paths_to_tensor(X_train).astype('float32')/255
test_tensors = paths_to_tensor(X_test).astype('float32')/255

In [5]:
## Benchmark Model ##
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

model = Sequential()

model.add(Conv2D(filters=16, kernel_size=2, strides=1, padding='same', activation='relu', input_shape=(600, 800, 2)))
model.add(MaxPooling2D(pool_size=2, padding='same'))
model.add(Conv2D(filters=32, kernel_size=2, strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2, padding='same'))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 600, 800, 16)      144       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 300, 400, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 300, 400, 32)      2080      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 150, 200, 32)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 960000)            0         
_________________________________________________________________
dense_1 (Dense)      

In [6]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
from keras.callbacks import ModelCheckpoint  

epochs = 5

### Do NOT modify the code below this line.

model.fit(train_tensors, y_train, epochs=epochs, batch_size=5, verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
116/116 [==============================] - 32s 278ms/step - loss: 7.7289 - acc: 0.4914
Epoch 2/5
116/116 [==============================] - 30s 257ms/step - loss: 7.9712 - acc: 0.5000
Epoch 3/5
116/116 [==============================] - 30s 255ms/step - loss: 7.9712 - acc: 0.5000
Epoch 4/5
116/116 [==============================] - 30s 255ms/step - loss: 7.9712 - acc: 0.5000
Epoch 5/5
116/116 [==============================] - 31s 265ms/step - loss: 7.9712 - acc: 0.5000


In [8]:
predictions = model.predict(test_tensors)
predictions = [round(float(i), 0) for i in predictions]
predictions

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0]

In [9]:
y_test

[1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0]

In [10]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, predictions)
print("The accuracy is : {}%".format(round(accuracy*100, 2)))

The accuracy is : 44.83%
